In [1]:
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim

/var/folders/95/0csrhky965x_1zggpg7jfkr00000gn/T/ipykernel_60317/1913194641.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
chords = pd.read_csv("output_chords.csv", header=None)
# chords.head

chords_sorted = chords.apply(lambda x: sorted(x), axis=1)

# Drop duplicates to get unique chords
# Count the unique chords
# vocab_size = len(chords_sorted.drop_duplicates())
# print("Vocabulary Size (Unique Chords):", vocab_size)
# # vocab is 10950 for /content/output_chords.csv
vocab_size = 10950


In [3]:
import pandas as pd
import numpy as np
import random

def create_sequences(df, sequence_length=4, chord_length=6, num_notes_to_select=5, random_seed=42, data_percent=0.01):
    input_sequences = []
    output_notes = []
    random.seed(random_seed)
    
    # Calculate the number of samples to select
    num_samples = int(len(df) * data_percent)
    
    # Ensure at least one sample is selected
    num_samples = max(1, num_samples)
    
    # Randomly select sample indices
    sampled_indices = random.sample(range(len(df) - sequence_length + 1), num_samples)
    
    for i in sampled_indices:
        sequence = df.iloc[i:i + sequence_length].values.tolist()

        # The last chord in the sequence
        fourth_chord = sequence[-1].copy()

        # Count non-zero notes in the chord
        non_zero_notes = [note for note in fourth_chord if note != 0]
        num_notes_in_chord = len(non_zero_notes)

        # Select the appropriate number of notes to output based on the number of notes in the chord
        if num_notes_in_chord > 1:
            output = random.sample(non_zero_notes, min(num_notes_to_select, num_notes_in_chord - 1))
        else:
            output = []

        # The remaining note to keep in the chord
        remaining_note = [note for note in non_zero_notes if note not in output]
        if not remaining_note:
            remaining_note = [0]

        # Ensure the last chord has one remaining note and is padded to chord_length
        last_chord = remaining_note + [0] * (chord_length - len(remaining_note))

        # The rest of the chords in the sequence remain the same
        input_sequence = sequence[:-1]
        input_sequence.append(last_chord)

        input_sequences.append(input_sequence)
        output_notes.append(output + [0] * (num_notes_to_select - len(output)))  # Pad output to ensure fixed length

    return np.array(input_sequences), np.array(output_notes)


# Example usage
data = {
    'note1': [60, 62, 60, 65, 65, 60],
    'note2': [64, 65, 64, 69, 64, 62],
    'note3': [67, 69, 67, 72, 72, 64],
    'note4': [0, 0, 72, 76, 0, 67],
    'note5': [0, 0, 0, 0, 0, 69],
    'note6': [0, 0, 0, 0, 0, 71]
}

df = pd.DataFrame(data)

input_sequences, output_notes = create_sequences(df, sequence_length=4, chord_length=6, num_notes_to_select=5)

print("Input Sequences:")
print(input_sequences)
print("Output Notes:")
print(output_notes)



Input Sequences:
[[[60 64 67 72  0  0]
  [65 69 72 76  0  0]
  [65 64 72  0  0  0]
  [62  0  0  0  0  0]]]
Output Notes:
[[60 71 64 69 67]]


In [4]:

import numpy as np
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch

batch_size=128

# Load the dataset
dataset = pd.read_csv('output_chords.csv')

# Create input-output pairs

input_sequences, output_notes = create_sequences(dataset)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, output_notes, test_size=0.2, random_state=42,shuffle=False)


class ChordDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence_matrix = self.sequences[idx]
        return {
            'sequence': torch.tensor(sequence_matrix, dtype=torch.long),
            'label': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Adjust the create_sequences function if needed to ensure correct formatting


# Creating data loaders for training and validation
train_dataset = ChordDataset(X_train, y_train)
val_dataset = ChordDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# The train_loader and val_loader are now ready to be used for training and validation


In [5]:


# Assuming the definition of the MusicTransformer model is available from model.py
# Initialize the Music Transformer model
note_vocab_size = 1024  # As specified earlier
embed_size = 384  # Example size, adjust as needed
num_layers = 6  # Example value, adjust as needed
heads = 6  # Example value, adjust as needed
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
forward_expansion = 4  # Example value, adjust as needed
dropout = 0.3 # Example dropout rate, adjust as needed
max_length = 100  # Maximum sequence length, adjust as needed
epochs=30
learning_rate= 0.0001

from model_v2 import NotePredictor

model = NotePredictor(
    input_dim=note_vocab_size,
    hidden_dim=embed_size,
    num_layers=num_layers,
    num_heads=heads,
    dropout=dropout,
    max_length=max_length,
    forward_expansion=forward_expansion
).to(device)

# for name, param in model.named_parameters():
#   print(f"{name}: {param.size()}")

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)


# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            optimizer.zero_grad()
            input_data = batch['sequence'].to(device)
            targets = batch['label'].to(device)
            # Ensure the tensor is of integer type
            targets = targets.long()  # Convert to long if not already an integer type


            batch_size, num_notes_to_select = targets.shape

            one_hot_encoded_columns = []
            for i in range(num_notes_to_select):
                one_hot_encoded_columns.append(F.one_hot(targets[:, i], num_classes=note_vocab_size))

            combined = torch.cat(one_hot_encoded_columns, dim=-1)
            one_hot_final = combined[:, :128]
            outputs = model(input_data)  # Assuming no mask for simplicity

            # Calculate the loss
            loss = criterion(outputs, one_hot_final.float())
            
            
            
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

        # Validation Loop
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                input_data = batch['sequence'].to(device)
                targets = batch['label'].to(device)
                targets = targets.long()
                
                
                    
                one_hot_encoded_columns = []
                for i in range(num_notes_to_select):
                    one_hot_encoded_columns.append(F.one_hot(targets[:, i], num_classes=note_vocab_size))

                combined = torch.cat(one_hot_encoded_columns, dim=-1)
                one_hot_final = combined[:, :128]
                outputs = model(input_data)  # Assuming no mask for simplicity


                val_loss = criterion(outputs, one_hot_final.float())
                # Here, you can also calculate accuracy or other metrics as needed

        print(f"Validation Loss: {val_loss.item():.4f}")

    # Save the model
    print('saving model_state_dict')

    hyperparameters = {
      'learning_rate': learning_rate,
      'batch_size': batch_size,
      'num_epochs': epochs,
      'embed_size': embed_size,
      'num_layers': num_layers,
      'heads': heads,
      'forward_expansion': forward_expansion,
      'dropout': dropout,
      'max_length': max_length,
      'note_vocab_size': note_vocab_size,
    }

    # Combine the model's state dict and the hyperparameters in a single dictionary
    save_dict = {
        'model_state_dict': model.state_dict(),
        'hyperparameters': hyperparameters
    }


    torch.save(save_dict, 'model_state_dict.pth')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)


/Users/omrinuri/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 